#### Attention:
Ceci ne prend pas en compte les départements d'outre mer mais uniquement la métrople !! (je n'ai pas trouvé de carte fonctionnel)

In [1]:
%pip install altair pandas numpy plotly
import altair as alt 
alt.data_transformers.enable('json')

Note: you may need to restart the kernel to use updated packages.


DataTransformerRegistry.enable('json')

In [2]:
# imports
import pandas as pd
# import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np

In [3]:
df = pd.read_csv("data/dpt2020.csv", sep=";")

# Vérifier si '2A' ou '2B' sont présents
print("'2A' présent ?", '2A' in df['dpt'].unique())
print("'2B' présent ?", '2B' in df['dpt'].unique())

'2A' présent ? False
'2B' présent ? False


In [4]:
print(f"Initial shape is {df.shape}")

# supp years and dpt = XXXX and transform to int
to_clean=['annais', 'dpt']
for col in to_clean:
    #df[col] = df[col].astype(str).str.strip()
    
    # Supprimer les lignes où la colonne n'est pas strictement numérique
    df = df[df[col].str.isnumeric()]
    
    if col == 'annais':
        df[col] = df[col].astype(int)
        
    else:
    # Reforcer le type string natif (facultatif mais propre)
        df[col] = df[col].astype("string")
        
        ### Ce n'est pas supported par la carte que nous avons ###
        # Transformation pour la corse => tout est transformé en 2A
        df.loc[df['dpt'] == '20', 'dpt'] = '2A'
        # Exclusion des DOM_TOM
        df = df[~df['dpt'].isin(['971', '972', '973', '974'])]
    print(df[col].dtype)
    print(f"Shape is now {df.shape}")

Initial shape is (3727553, 5)
int64
Shape is now (3690309, 5)
string
Shape is now (3492154, 5)


In [5]:
# Supprimer les prénoms "_PRENOMS_RARES"
df = df[df["preusuel"] != "_PRENOMS_RARES"]
# Ajouter une colonne de longueur de prénom
df["longueur"] = df["preusuel"].str.len()
# Supprimer les prénoms de longueur 1
df = df[df["longueur"] > 1]
print(f"Shape is now {df.shape}")

Shape is now (3471072, 6)


In [6]:
df

,sexe,preusuel,annais,dpt,nombre,longueur
10885,1,AADIL,1983,84,3,5
10886,1,AADIL,1992,92,3,5
10888,1,AAHIL,2016,95,3,5
10892,1,AARON,1962,75,3,5
10893,1,AARON,1976,75,3,5
...,...,...,...,...,...,...
3727543,2,ZYA,2011,91,3,3
3727545,2,ZYA,2013,44,4,3
3727546,2,ZYA,2013,59,3,3
3727548,2,ZYA,2018,59,3,3


In [7]:
max_nombre = df.groupby(['sexe', 'annais', 'dpt'])['nombre'].transform('max')
df_final = df[df['nombre'] == max_nombre]
df_final = df_final.sort_values(by=['sexe', 'annais', 'dpt'])
moyenne_longueur = df.groupby(['sexe', 'annais', 'dpt'])['longueur'].transform('mean').round().astype(int)
df_final['moyenne_longueur'] = moyenne_longueur

In [8]:
df_final

,sexe,preusuel,annais,dpt,nombre,longueur,moyenne_longueur
770685,1,JEAN,1900,01,144,4,6
114883,1,ANDRÉ,1900,02,138,5,6
770687,1,JEAN,1900,03,230,4,6
1023835,1,LOUIS,1900,04,40,5,6
900377,1,JOSEPH,1900,05,50,6,6
...,...,...,...,...,...,...,...
2864122,2,LINA,2020,91,69,4,5
2377520,2,EMMA,2020,92,132,4,6
2864124,2,LINA,2020,93,101,4,6
2864125,2,LINA,2020,94,87,4,6


In [9]:
grouped = df.groupby(['annais', 'dpt', 'sexe', 'preusuel'], as_index=False)['nombre'].sum()
top_names = grouped.sort_values('nombre', ascending=False).groupby(['annais', 'dpt', 'sexe']).first().reset_index()

In [10]:
top_names = top_names.rename(columns={'preusuel': 'prenom'})

In [11]:
top_names['dpt'] = top_names['dpt'].str.zfill(2)

In [12]:
top_names

,annais,dpt,sexe,prenom,nombre
0,1900,01,1,JEAN,144
1,1900,01,2,MARIE,639
2,1900,02,1,ANDRÉ,138
3,1900,02,2,MARIE,319
4,1900,03,1,JEAN,230
...,...,...,...,...,...
22305,2020,93,2,LINA,101
22306,2020,94,1,ADAM,136
22307,2020,94,2,LINA,87
22308,2020,95,1,ADAM,133


In [13]:
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
departements = alt.topo_feature(url, "features")

In [14]:
### TEST POUR LES CORRESPONDANCES DES DEPARTEMENTS ###
import requests
response = requests.get(url)
geojson = response.json()

# Extraire tous les codes de département du GeoJSON
geojson_codes = {feature['properties']['code'] for feature in geojson['features']}

# Extraire les codes du dataset
top_names_codes = set(top_names['dpt'].unique())

print("Codes présents dans GeoJSON mais absents de top_names :", geojson_codes - top_names_codes)
print("Codes présents dans top_names mais absents du GeoJSON :", top_names_codes - geojson_codes)
print(f"Correspondances exactes : {len(geojson_codes & top_names_codes)} / {len(geojson_codes)}")

Codes présents dans GeoJSON mais absents de top_names : {'2B'}
Codes présents dans top_names mais absents du GeoJSON : set()
Correspondances exactes : 95 / 96


In [15]:
import altair as alt

# Paramètre pour l'année (avec un slider)
year_param = alt.param(
    name="année",
    bind=alt.binding_range(
        min=top_names['annais'].min(),
        max=top_names['annais'].max(),
        step=1,
        name="Année :"
    ),
    value=2000
)

# Paramètre pour le sexe
gender_param = alt.param(
    name="sexe",
    bind=alt.binding_radio(
        options=['1', '2'],
        labels=['♂ Homme', '♀ Femme'],
        name="Sexe :"
    ),
    value='1'
)

# Carte choroplèthe avec lookup + filtrage
carte = alt.Chart(departements).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).encode(
    tooltip=[
        alt.Tooltip('properties.nom:N', title='Département'),
        alt.Tooltip('prenom:N', title='Prénom')
    ],
    color=alt.Color('prenom:N', title='Prénom populaire', legend=None)
).transform_lookup(
    lookup='properties.code',
    from_=alt.LookupData(
        top_names,
        key='dpt',
        fields=['preusuel'] # ['annais', 'sexe', 'preusuel']
    )
).transform_filter(
    (alt.datum.annais == year_param) & (alt.datum.sexe == gender_param)
).project(
    type='mercator'
).add_params(
    year_param,
    gender_param
).properties(
    width=700,
    height=600,
    title='Carte interactive des prénoms les plus populaires par département'
)

carte


alt.Chart(...)

In [16]:
df_final[df_final.dpt == "02"]

,sexe,preusuel,annais,dpt,nombre,longueur,moyenne_longueur
114883,1,ANDRÉ,1900,02,138,5,6
1096195,1,MARCEL,1901,02,123,6,6
1096289,1,MARCEL,1902,02,158,6,6
1096383,1,MARCEL,1903,02,157,6,6
115256,1,ANDRÉ,1904,02,166,5,6
...,...,...,...,...,...,...,...
2909458,2,LOUISE,2016,02,42,6,6
2640988,2,JADE,2017,02,45,4,5
2909653,2,LOUISE,2018,02,42,6,5
2377332,2,EMMA,2019,02,42,4,5


In [31]:
import geopandas as gpd
from io import BytesIO
import requests

url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
response = requests.get(url)
response.raise_for_status()
gdf = gpd.read_file(BytesIO(response.content))
gdf["geometry"] = gdf["geometry"].simplify(0.01)

dep = gdf.merge(df_final, how='right', left_on='code', right_on='dpt')
dep['moyenne_longueur'] = dep['moyenne_longueur'].fillna(0).astype(int)
dep = dep.dropna(subset=['geometry'])

# 5. Créer le slider sur la colonne 'longueur'
slider = alt.binding_range(min=dep['moyenne_longueur'].min(), max=dep['moyenne_longueur'].max(), step=1)
selection = alt.selection_point(
    name='longueur',
    fields=['moyenne_longueur'], 
    bind=slider, 
    value=dep['moyenne_longueur'].min()
)

In [35]:
carte = alt.Chart(dep).mark_geoshape(
    stroke='white'
).encode(
    color=alt.condition(
        selection,
        alt.Color("moyenne_longueur:Q", scale=alt.Scale(scheme='blues'), title="Longueur"),
        alt.value("lightgray")  # couleur par défaut
    ),
    tooltip=["nom:N", "code:N", "moyenne_longueur:Q"]
).add_params(
    selection
).project(
    type='mercator'
).properties(
    width=600,
    height=600
)
carte

alt.Chart(...)